In [1]:
import torch
from PIL import Image
import torchvision.transforms as transforms
import numpy as np
from timeit import default_timer as timer
import os


img_transforms = transforms.Compose(
    [
        transforms.Resize((768, 768), transforms.InterpolationMode.NEAREST),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)), # remove?
    ],
)

# trt, amp, 256, 1 channel


In [ ]:
generator = define_G(
    input_nc=3,
    output_nc=3,
    ngf=64,
    netG="unet_256",
    norm="batch",
)

generator.cuda()
generator.eval()

state_dict = torch.load("/raid/colon_reproduce/ct_us_cyclegan/checkpoints/mask2us_liver_add_conv_pad_resize/1340_net_G.pth")
generator.load_state_dict(state_dict)

In [3]:
root_dir = "/raid/colon_reproduce/ct_us_cyclegan/datasets/mask2us_liver_3cls/B/test/"
test_list = os.listdir(root_dir)

In [ ]:
for test_name in test_list:
    start = timer()
    test_image = Image.open(os.path.join(root_dir, test_name)).convert("RGB")
    test_input = img_transforms(test_image).unsqueeze(0).cuda()

    load_end = timer()

    with torch.no_grad():
        output = generator(test_input)

    output = output.squeeze(0)
    output = output.permute(1, 2, 0)
    output = (output + 1) / 2.0 * 255.0

    output = output.cpu().float().numpy().astype(np.uint8)
    end = timer()
    print(f"Time to load: {load_end - start } generate: {end - load_end} total: {end - start} shape: {output.shape}")